<a href="https://colab.research.google.com/github/martinlee192/AI-basic/blob/master/simple_layer_X_ray_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)
print(device)

cuda


In [5]:
from google.colab import drive
drive.mount('/gdrive')
!dir /gdrive/MyDrive/data

Mounted at /gdrive
x_ray_test_data  x_ray_test_data.zip  x_ray_train_data


In [6]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='/gdrive/MyDrive/data/x_ray_train_data', transform=trans)

In [7]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),#224-5+1 = 220
            nn.ReLU(),
            nn.MaxPool2d(2), # 110
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5), #110-5+1 = 1
            nn.ReLU(),
            nn.MaxPool2d(2),#58 
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*53*53, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)           
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.shape[0], -1)
        #print(out.shape)
        out = self.layer3(out)
        return out
        

In [9]:
#Test
net = CNN().to(device)
test_input = (torch.Tensor(3,3,224,224)).to(device)
test_out = net(test_input)

In [10]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [11]:
total_batch = len(data_loader)

epochs = 20
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')  

[Epoch:1] cost = 0.2780904173851013
[Epoch:2] cost = 0.13469533622264862
[Epoch:3] cost = 0.1176791563630104
[Epoch:4] cost = 0.10116229206323624
[Epoch:5] cost = 0.09096332639455795
[Epoch:6] cost = 0.08385280519723892
[Epoch:7] cost = 0.07241908460855484
[Epoch:8] cost = 0.06812445819377899
[Epoch:9] cost = 0.06025095656514168
[Epoch:10] cost = 0.05087011307477951
[Epoch:11] cost = 0.047803282737731934
[Epoch:12] cost = 0.04397217929363251
[Epoch:13] cost = 0.035734307020902634
[Epoch:14] cost = 0.03116600774228573
[Epoch:15] cost = 0.024433951824903488
[Epoch:16] cost = 0.023536086082458496
[Epoch:17] cost = 0.022982995957136154
[Epoch:18] cost = 0.014400738291442394
[Epoch:19] cost = 0.013335677795112133
[Epoch:20] cost = 0.012451263144612312
Learning Finished!


In [12]:
torch.save(net.state_dict(),"/gdrive/MyDrive/data/model_simple_layer_20epoch.pth")

In [13]:
new_net = CNN().to(device)

In [14]:
new_net.load_state_dict(torch.load('/gdrive/MyDrive/data/model_simple_layer_20epoch.pth'))

<All keys matched successfully>

In [16]:
test_data = torchvision.datasets.ImageFolder(root='/gdrive/MyDrive/data/x_ray_test_data', transform=trans)

In [17]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [18]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.7644230723381042
